In [1]:
import plotly.express as px
import pandas as pd
from lib.utils import load_delta_from_dir_by_index,run_old_state_machine_on_thresholded_predictions,run_new_state_machine_on_thresholded_predictions,forward_casey,forward_casey_corrected
from lib.models import MLP
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

ImportError: cannot import name 'load_delta_from_dir_by_index' from 'lib.utils' (/home/andrew/smoking/smoking_ml/lib/utils.py)

In [99]:
# recreate old pipeline
rawlabel_old = forward_casey(X)
df['rawlabel_old'] = rawlabel_old
df['rawlabel_old_10'] = df['rawlabel_old']*10
# proof that recomputed rawlabels and watch rawlabels are the same
# fig = px.line(df[:70000],x='index',y=['acc_x','rawlabel_10','rawlabel_old_10'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
# fig.show(renderer='browser')
# do thresholding
label_old = [1 if y > .85 else 0 for y in rawlabel_old]
df['label_old'] = label_old
df['label_old_10'] = df['label_old']*10
# thresholds will obviously be the same, but we can check
# fig = px.line(df[:70000],x='index',y=['acc_x','label_10','label_old_10'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
# fig.show(renderer='browser')
# run old state machine
states_old,puff_locations_old = run_old_state_machine_on_thresholded_predictions(label_old)
df['state_old'] = states_old
# fig = px.line(df[:70000],x='index',y=['acc_x','state','state_old'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
# fig.show(renderer='browser')
# let's plot puff locations
fig = px.line(df[:70000],y=['acc_x','label_old_10','state_old'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc)
fig.show(renderer='browser')

100%|██████████| 16191/16191 [00:00<00:00, 20971.80it/s]


In [104]:
# run thrasher data through new network
rawlabel = forward_casey_corrected(X)
df['rawlabel'] = rawlabel
df['rawlabel_10'] = df['rawlabel']*10
# thresholding
label = [1 if y > .85 else 0 for y in rawlabel]
df['label'] = label
df['label_10'] = df['label']*10
# state machine
states,puff_locations = run_new_state_machine_on_thresholded_predictions(label)
df['state'] = states
# compare old and new
fig = px.line(df[:70000],y=['acc_x','rawlabel_10','label_10','state'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_color="green")
# for puff_loc in puff_locations_old:
#     fig.add_vline(x=puff_loc, line_color="red",line_dash="dash",line_width=2)
fig.show(renderer='browser')

100%|██████████| 112519/112519 [00:10<00:00, 10693.54it/s]


In [54]:
# session detection
puff_signal = np.zeros(len(df))
puff_signal[np.array(puff_locations_old)] = 1
puffs_in_session = []
session_count = 0
sessions = []
puffs = []
puff_count = 0
session = False
in_session = []
puff_counts = []
for i,_ in enumerate(puff_signal):
    puffs = [puff - 1 for puff in puffs]
    if (session):
        if(puffs[0] == 0):
            # meaning the earliest puff in the session has now expired
            session = False
            puffs = []
        else:
            puffs_in_session.append(len(puffs))
            puff_counts.append(puff_count)
            sessions.append(session_count)
            in_session.append(1)
            if(puff_signal[i]):
                puffs.append(9600)
            continue
    puffs = [puff for puff in puffs if puff > 0]
    if(puff_signal[i]):
        puff_count += 1
        puffs.append(9600)
    if(puff_count == 3):
        puff_count = 0
        session_count += 1
        session = True
        
    puffs_in_session.append(len(puffs))
    in_session.append(0)
    puff_counts.append(puff_count)
    sessions.append(session_count)
df['puff_count_old'] = puffs_in_session
df['puff_counts'] = puff_counts
df['sessions'] = sessions
df['in_session'] = in_session
df['in_session'] = df['in_session']*10

fig = px.line(df[:70000],y=['acc_x','puff_count_old','puff_counts','sessions','in_session'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc, line_color="red",line_width=1)
fig.show(renderer='browser')

In [105]:
# session detection
puff_signal = np.zeros(len(df))
puff_signal[np.array(puff_locations)] = 1
puffs_in_session = []
session_count = 0
sessions = []
puffs = []
puff_count = 0
session = False
in_session = []
puff_counts = []
for i,_ in enumerate(puff_signal):
    puffs = [puff - 1 for puff in puffs]
    if (session):
        if(puffs[0] == 0):
            # meaning the earliest puff in the session has now expired
            session = False
            puffs = []
        else:
            puffs_in_session.append(len(puffs))
            puff_counts.append(puff_count)
            sessions.append(session_count)
            in_session.append(1)
            if(puff_signal[i]):
                puffs.append(9600)
            continue
    puffs = [puff for puff in puffs if puff > 0]
    if(puff_signal[i]):
        puff_count += 1
        puffs.append(9600)
    if(puff_count == 3):
        puff_count = 0
        session_count += 1
        session = True
        
    puffs_in_session.append(len(puffs))
    in_session.append(0)
    puff_counts.append(puff_count)
    sessions.append(session_count)
df['puff_count_old'] = puffs_in_session
df['puff_counts'] = puff_counts
df['sessions'] = sessions
df['in_session'] = in_session
df['in_session'] = df['in_session']*10

fig = px.line(df[:70000],y=['acc_x','puff_count_old','puff_counts','sessions','in_session'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc, line_color="red",line_width=1)
fig.show(renderer='browser')

In [ ]:

for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_dash="dash", line_color="green",line_width=.3)
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc, line_color="red",line_width=.3)
for session_loc in session_idx:
    fig.add_vline(x=session_loc,line_dash="dash", line_color="green",line_width=2)
for session_loc in session_idx_old:
    fig.add_vline(x=session_loc, line_color="red",line_width=2)
fig.show(renderer='browser')

In [ ]:
# recreate new pipeline
rawlabel = forward_casey_corrected(X)
# verify
# rawlabel == df['rawlabel']
# np.allclose(rawlabel[:19004],df['rawlabel'][:19004])
# np.allclose(rawlabel,df['rawlabel'])
# plt.plot(rawlabel[-100:])
# plt.plot(df['rawlabel'].to_numpy()[-100:])
# thresholding
label = [1 if y > .85 else 0 for y in rawlabel]
(label == df['label']).all() # works because integer-valued
# state machine
states,puff_locations = run_new_state_machine_on_thresholded_predictions(label)
(states == df['state']).all() # works because integer-valued
# compare old and new
fig = px.line(df,y=['acc_x','label_10','state','label_old_10','state_old'],labels={'index':'time (seconds)','value':'acceleration (m/s^2)'})
for puff_loc in puff_locations:
    fig.add_vline(x=puff_loc,line_color="green")
for puff_loc in puff_locations_old:
    fig.add_vline(x=puff_loc, line_color="red",line_dash="dash",line_width=2)
fig.show(renderer='browser')